PSEUDO CODE:

Algorithm: REINFORCE
Initialise parameters: step size 𝛼 ∈ (0,1]
Initialise policy network 𝜋 with parameters 𝜽

For episode = 1, 𝑀 do
    Generate an episode trajectory 𝜏~𝜋𝜽
    For 𝑡 = 1, 𝑇 − 1 do
       G <- Sum of rewards
       𝜽 <- update parameters
    End For
End For



In [4]:
import gym
import numpy as np
from collections import deque
import torch 
import torch.nn as nn

# Initialize the environment
env = gym.make('CartPole-v1')

# Define the policy network
class PolicyNetwork(nn.Module):
    def __init__(self, state_size, action_size, hidden_size=32):
        super(PolicyNetwork, self).__init__()
        self.fc1 = nn.Linear(state_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, action_size)

    def forward(self, state):
        x = torch.relu(self.fc1(state))
        action_probs = torch.softmax(self.fc2(x), dim=-1)
        return action_probs

# Reinforce algorithm
def reinforce(num_episodes, max_steps_per_episode, gamma=0.99, lr=0.01):
    policy_net = PolicyNetwork(state_size=env.observation_space.shape[0],
                               action_size=env.action_space.n)
    optimizer = torch.optim.Adam(policy_net.parameters(), lr=lr)
    
    for episode in range(num_episodes):
        state = env.reset()

        state = state[0]
        episode_rewards = deque()

        
        for step in range(max_steps_per_episode):
            action_probs = policy_net(torch.from_numpy(state).float())
            action = np.random.choice(env.action_space.n, p=action_probs.detach().numpy())
            
            next_state, reward, done, _ = env.step(action)
            episode_rewards.append(reward)
            
            if done:
                break
                
            state = next_state
        
        # Compute the discounted cumulative reward
        discounted_rewards = []
        R = 0
        for r in episode_rewards[::-1]:
            R = r + gamma * R
            discounted_rewards.insert(0, R)
        
        # Normalize the discounted rewards
        discounted_rewards = torch.tensor(discounted_rewards)
        discounted_rewards = (discounted_rewards - discounted_rewards.mean()) / (discounted_rewards.std() + 1e-9)
        
        # Compute the policy gradient and update the policy network
        policy_losses = []
        for reward, log_prob in zip(discounted_rewards, log_probs):
            policy_losses.append(-log_prob * reward)
        
        policy_loss = torch.stack(policy_losses).sum()
        optimizer.zero_grad()
        policy_loss.backward()
        optimizer.step()
        
    return policy_net

# Train the policy network
policy_net = reinforce(num_episodes=1000, max_steps_per_episode=200)





/Users/luisweekes/anaconda3/envs/myenv/lib/python3.11/site-packages/gym/utils/passive_env_checker.py:233: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(terminated, (bool, np.bool8)):


ValueError: too many values to unpack (expected 4)